In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima

import torch
import torch.nn as nn
import torch.nn.functional as F

from metric import mape, mae

In [15]:
# data read
train_df = pd.read_csv('./data/train.csv', encoding='cp949')
test_df = pd.read_csv('./data/pre_test01.csv', encoding='cp949')
submission = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [3]:
train_df

,num,date_time,elec_amount,temp,wind_speed,humidity,rainfall,sunshine,no_elec,sunlight_have
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1


In [8]:
def df2d_to_array3d(df_2d):
    feature_size=df_2d.iloc[:,2:].shape[1]
    time_size=len(df_2d['date_time'].value_counts())
    sample_size=len(df_2d.num.value_counts())
    return df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])

In [9]:
train_df=df2d_to_array3d(train_df)
test_df=df2d_to_array3d(test_df)

In [10]:
train_df.shape, test_df.shape

((60, 2040, 8), (60, 168, 7))

In [13]:
valid_pred_array=np.zeros([60, 168])
for idx in range(train_df.shape[0]):
    x_series=train_df[idx, :, 0]
    model=auto_arima(x_series, start_p=0, start_q=0, max_p=15, max_d=3, max_q=15, trace=False)
    preds=model.predict(1, 168, typ='levels')
    valid_pred_array[idx, :]=preds


In [14]:
valid_pred_array

array([[8702.43633721, 8702.43633721, 8702.43633721, ..., 8702.43633721,
        8702.43633721, 8702.43633721],
       [1108.99068808, 1108.99068808, 1108.99068808, ..., 1108.99068808,
        1108.99068808, 1108.99068808],
       [3447.98240541, 3447.98240541, 3447.98240541, ..., 3447.98240541,
        3447.98240541, 3447.98240541],
       ...,
       [ 646.80223225,  646.80223225,  646.80223225, ...,  646.80223225,
         646.80223225,  646.80223225],
       [ 814.74420673,  814.74420673,  814.74420673, ...,  814.74420673,
         814.74420673,  814.74420673],
       [3154.80033351, 3154.80033351, 3154.80033351, ..., 3154.80033351,
        3154.80033351, 3154.80033351]])

In [17]:
submission['answer']=valid_pred_array.reshape([-1,1])
submission.to_csv('submission.csv', index=False)